In [1]:
from dataclasses import dataclass

@dataclass # This decorator helps save time from initializing constructor
class Flash_card:
    question: str
    demo: str
    choices: list
    correct_choice: list
        
    # Shuffle all choices
    def shuffle_choices(self):
        np.random.shuffle(self.choices)   

In [2]:
from termcolor import colored
import numpy as np

class Folder:
    my_flash = list()
    correct = 0
        
    def learn(self):
        # Shuffle order of questions
        np.random.shuffle(self.my_flash)
        for i, flash in enumerate(self.my_flash, 1):
            print(colored(f'Q{i}. {flash.question}', 'blue'))
            if flash.demo:
                print(flash.demo.strip())
            # Shuffle choices
            flash.shuffle_choices()
            for index, c in enumerate(flash.choices):
                print(colored(f'{chr(index+97)}. {c}', 'magenta'))
            answers = []
            for j, _ in enumerate(flash.correct_choice, 1):
                answer = input(f'Choose answer {j} of {len(flash.correct_choice)}')
                # User chooses a, b, c, etc. instead of number
                while ord(answer) - 97 < 0 or ord(answer) - 97 >= len(flash.choices):
                    answer = input(f'Choose your answer between {chr(97)} and {chr(96 + len(flash.choices))}')
                answers.append(flash.choices[ord(answer) - 97])
            if sorted(answers) == sorted(flash.correct_choice):
                print('Correct!')
                self.correct += 1
            else:
                print(colored(f'Wrong! Correct choice should be: ', 'yellow'), end='')
                print(colored('\n'.join(flash.correct_choice), 'red'))
            print()
        print(f'You get {self.correct * 100 / len(self.my_flash):.2f}%')

In [3]:
def read_a_file(file_path: str) -> Folder:
    doc = Folder()
    with open(file_path, 'r') as f:
        q, demo, correct_choice, choices = '', '', [], []
        for l in f:
            if l.startswith('####'):
                # Add question
                q = l[l.find('.')+1:].strip()
            elif l[:3] not in ['###', '- [', '\n', '\r\n']:
                demo += l.strip() + '\n'
            elif l.startswith('- ['):
                if l.startswith('- [x'):
                    correct_choice.append(l[6:].strip())
                choices.append(l[5:].strip())
            # Detect a blank line
            elif l in ['\n', '\r\n']:
                a_flash_card = Flash_card(question=q, demo=demo, choices=choices, correct_choice=correct_choice)
                doc.my_flash.append(a_flash_card)
                q, demo, correct_choice, choices = '', '', [], []
    return doc

In [4]:
def create_a_nicer_file(doc: Folder, filename: str):
    '''
    This is to create a nicer Q&A file name from doc a Folder instance
    '''
    with open(str(filename), 'w') as f:
        for i, flash in enumerate(doc.my_flash, 1):
            f.write(f'####Q{i}. {flash.question}\n') # Write question
            if flash.demo != '': # Write demo if it exists
                f.write(flash.demo)
            flash.shuffle_choices() # Shuffle choices
            for choice in flash.choices:
                if choice in flash.correct_choice:
                    f.write(f'- [x] {choice}')
                else:
                    f.write(f'- [ ] {choice}')
            f.write('\n')

In [ ]:
doc = read_a_file('Git.txt')
# create_a_nicer_file(doc, 'NoSQL1.txt')
doc.learn()

Q1. Which option enables inclusion of author name in custom log format?
a. %am
b. %ae
c. %ad
d. %an


Choose answer 1 of 1 a


Wrong! Correct choice should be: %an

Q2. Why would the following command be used?
`git rebase -i HEAD~10`
a. In order to locally cache the last 10 commits
b. To run a comparative search of the last 10 commits for differences
c. To list the last 10 commits and modify them with either the squash or fixup command
d. To delete the last 10 commits and reset the HEAD


Choose answer 1 of 1 b


Wrong! Correct choice should be: To list the last 10 commits and modify them with either the squash or fixup command

Q3. Which of the following is called dumb protocol?
a. Git
b. SSH
c. HTTP
d. All of these


Choose answer 1 of 1 c


Correct!

Q4. What conflicts can occur when forcing a push after rebasing?
a. The current HEAD will be deleted and can't be reinstated.
b. The origin URL will be reset to its default value.
c. Nothing, it's common practice to force a push after rebasing.
d. The remote master branch could have existing changes overwritten.


Choose answer 1 of 1 d


Correct!

Q5. After starting to merge a feature branch into your master branch, you encounter a merge conflict and decide you do not want to perform the merge. How can you stop the merge and restore to the pre-merge state?
a. Use git restore -p.
b. Use git merge -u.
c. Use git merge --undo.
d. Use git merge --abort.
